# DDPG algorithm

## Initialize actor and critic network

In [34]:
import jax.numpy as jnp
import jax
from jax import grad, jit, vmap
from jax import random
from flax import linen as nn  # Linen API
from collections import deque
import gymnasium as gym

env = gym.make("MountainCarContinuous-v0")
seed = 0
key = random.PRNGKey(seed)

F_CPP_MIN_LOG_LEVEL=0
action_dim = env.action_space.shape[0]
state_dim = env.observation_space.shape[0]

print(action_dim)
print(state_dim)

1
2


In [35]:
#create the actor and critic newtorks like multilayer perceptrons


class Critic(nn.Module):
    """critic model MLP"""
    
    @nn.compact
    def __call__(self, x):
        x = nn.Dense(features=256)(x)
        x = nn.relu(x)
        x = nn.Dense(features=256)(x)
        x = nn.relu(x)
        x = nn.Dense(features=1)(x)
        return x
    
class Actor(nn.Module):
    """actor model MLP"""
    
    @nn.compact
    def __call__(self, x):
        x = nn.Dense(features=256)(x)
        x = nn.relu(x)
        x = nn.Dense(features=256)(x)
        x = nn.relu(x)
        x = nn.Dense(features=action_dim)(x)
        x = nn.tanh(x)
        return x

In [36]:
# Randomly initialize critic network Q(s, a|θ_Q ) and actor μ(s|θ_μ ) with weights θ_Q and θ_μ .
critic = Critic()
critic_input = jnp.ones((1, state_dim + action_dim))
critic_params = critic.init(key, critic_input)
actor = Actor()
actor_input = jnp.ones((1, state_dim))
actor_params = actor.init(key, actor_input)

print(actor.tabulate(key, (1, state_dim) ))

check_critic = jax.tree_util.tree_map(lambda x: x.shape, critic_params) #checking critic params
check_actor = jax.tree_util.tree_map(lambda x: x.shape, actor_params) #checking actor params

print("critic parameters:\n", check_critic, "\n")
print("actor parameters:\n", check_actor)


                                Actor Summary                                
┏━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ path    ┃ module ┃ inputs       ┃ outputs      ┃ params                   ┃
┡━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         │ Actor  │ - 1          │ float32[1]   │                          │
│         │        │ - 2          │              │                          │
├─────────┼────────┼──────────────┼──────────────┼──────────────────────────┤
│ Dense_0 │ Dense  │ - 1          │ float32[256] │ bias: float32[256]       │
│         │        │ - 2          │              │ kernel: float32[2,256]   │
│         │        │              │              │                          │
│         │        │              │              │ 768 (3.1 KB)             │
├─────────┼────────┼──────────────┼──────────────┼──────────────────────────┤
│ Dense_1 │ Dense  │ float32[256] │ float32[256] │ bias: float3

In [37]:
#foward example with the initial  parameters 
# the parameters never store in the model
critic_forward = critic.apply(critic_params, critic_input)
actor_forward = actor.apply(actor_params, actor_input)

print("critic forward test:", critic_forward, "\n")
print("actor forward test:", actor_forward)

critic forward test: [[0.13340816]] 

actor forward test: [[-0.4034107]]


In [38]:
# Initialize target network Q_0_target and μ_0_target with weights 
# θ_Q_target ← θ_Q , θ_μ_target ← θ_μ
target_critic = Critic()
target_actor = Actor()

target_critic_params = critic_params
target_actor_params = actor_params


In [39]:
# Initialize replay buffer R
buffer_size = 1000
buffer = deque(maxlen=buffer_size)

In [44]:
# episode loop

# Initialize a random process N for action exploration
def noise(noise_scale=0.1):
    return noise_scale * jax.random.normal(key, (action_dim,))
# Receive initial observation state s 1
state, info = env.reset(seed=seed)
print(state)
N = noise(0.1)
print(N)

    

[-0.47260767  0.        ]
[-0.02058423]


In [56]:
# Select action a_t = μ(s t |θ μ ) + N t according to the current policy and exploration noise
action = noise() + actor.apply(actor_params, observation)
# action = env.action_space.sample()
print("action:", action, "\n")


# Execute action a t and observe reward r t and observe new state s t+1
next_state, reward, terminated, truncated, info = env.step(action)
print("next state:", next_state, "\n")

# Store transition (s t , a t , r t , s t+1 ) in R
transition = (state, action, reward, next_state)
buffer.append(transition)
print("buffer:", buffer, "\n")

# Sample a random minibatch of N transitions (s i , a i , r i , s i+1 ) from R
batch_size = 1
indices = jax.random.choice(key, len(buffer), shape=(batch_size,), replace=False)
minibatch = [buffer[i] for i in indices]
print("minibatch:", minibatch)


action: [-0.07490155] 

next state: [-0.5075476  -0.00492303] 

buffer: deque([(array([-0.47260767,  0.        ], dtype=float32), Array([-0.07490155], dtype=float32), -0.0005610242343437654, array([-0.47310096, -0.0004933 ], dtype=float32)), (array([-0.47260767,  0.        ], dtype=float32), Array([-0.07490155], dtype=float32), -0.0005610242343437654, array([-0.4740839 , -0.00098293], dtype=float32)), (array([-0.47260767,  0.        ], dtype=float32), Array([-0.07490155], dtype=float32), -0.0005610242343437654, array([-0.4755492 , -0.00146528], dtype=float32)), (array([-0.47260767,  0.        ], dtype=float32), Array([-0.07490155], dtype=float32), -0.0005610242343437654, array([-0.47748595, -0.00193676], dtype=float32)), (array([-0.47260767,  0.        ], dtype=float32), Array([-0.07490155], dtype=float32), -0.0005610242343437654, array([-0.47987983, -0.00239386], dtype=float32)), (array([-0.47260767,  0.        ], dtype=float32), Array([-0.07490155], dtype=float32), -0.000561024234343